<a href="https://colab.research.google.com/github/BranMwangi/BranMwangi/blob/main/ADS2_7PAM2001_0901_Assignment_1_Deep_Learning_with_Keras_1__730912358.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 1 - Deep Learning with Keras

This assignment will test you on the following skills: implementing a convolutional neural network in Keras, utilising the Functional Model API, building custom layers and loss functions, and analysing a trained model.

The model you will be building is called a Variational Autoencoder (VAE), a special type of neural network that uses Bayesian Inference to generate synthetic data. As described in the lectures, a VAE compresses a sample input into a low dimensional space (a latent vector). A constraint is applied in the form of a modification to the loss function, which has the effect of forcing the latent vector to look like a standard normal distribution.

This notebook is divided into sections, which you should use to complete the following tasks:

1. Implement a custom layer that performs the "reparameterization trick" described in the lectures.
2. Use the functional model API in keras to create an encoder model and a decoder model, using the specifications provided. Combine these models into the full VAE architecture.
3. Train the model using the celeb_a dataset of celebrity faces.
4. Create plots that demonstrate the ability of the network to reconstruct images, and generate new images.

In addition to submitting your completed notebook, you should write a 1-page report that discusses the results of the model.

In [1]:
# Module imports

import tensorflow as tf
import pandas as pd
import numpy as np
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

# Task 1 - Reparameterisation layer

In [2]:
### Create a class called latent_sampling, which subclasses layers.Layer.
### The class should perform the reparameterisation trick in its .call()
### method.
### Reparameterization Trick: z = mean + epsilon * exp(ln(variance) * 0.5)
### epsilon = N(0,1), a unit normal with same dims as mean and variance

# Include the follow two lines in your .call method:
# self.add_loss(-0.5 * tf.reduce_sum(1 + logvar - tf.square(mean) - tf.exp(logvar)))
# self.add_metric(-0.5 * tf.reduce_sum(1 + logvar - tf.square(mean) - tf.exp(logvar)), name='kl_loss')

class latent_sampling(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super(latent_sampling, self).__init__(**kwargs)

    def call(self, mean, logvar):
        # Sample noise from a standard normal distribution
        epsilon = tf.random.normal(shape=tf.shape(mean))
        # Perform the reparameterization trick
        z = mean + epsilon * tf.exp(logvar * 0.5)
        # Add the KL divergence loss to the model
        self.add_loss(-0.5 * tf.reduce_sum(1 + logvar - tf.square(mean) - tf.exp(logvar)))
        self.add_metric(-0.5 * tf.reduce_sum(1 + logvar - tf.square(mean) - tf.exp(logvar)), name='kl_loss')
        return z




# Task 2 - Model Definitions

In [3]:
### Create the encoder model, using the functional API and the architecture
### detailed below. Use tf.keras.models.Model to initialise the model.

# Model: "encoder"
# ____________________________________________________________________________________________________
#  Layer (type)            Output Shape           Activation  kernel_size  padding  Input
# ====================================================================================================
#  enc_input (InputLayer)  [(None, 128, 128, 3)]  None
#  enc_conv_1 (Conv2D)     (None, 64, 64, 32)     ReLU        (3,3)        'same'   enc_input
#  enc_conv_2 (Conv2D)     (None, 32, 32, 64)     ReLU        (3,3)        'same'   enc_conv_1
#  enc_conv_3 (Conv2D)     (None, 16, 16, 64)     ReLU        (3,3)        'same'   enc_conv_2    
#  enc_conv_4 (Conv2D)     (None, 8, 8, 64)       ReLU        (3,3)        'same'   enc_conv_3    
#  enc_flat (Flatten)      (None, 4096)           None        None         None     enc_conv_4
#  z_mean (Dense)          (None, 200)            None        None         None     enc_flat                        
#  z_log_var (Dense)       (None, 200)            None        None         None     enc_flat
#  z (latent_sampling)     (None, 200)            None        None         None     (z_mean, z_log_var)

# Define the input layer
enc_input = tf.keras.layers.Input(shape=(128, 128, 3))

# Add a sequence of convolutional layers
x = tf.keras.layers.Conv2D(32, 3, activation='relu', padding='same')(enc_input)
x = tf.keras.layers.Conv2D(64, 3, activation='relu', padding='same')(x)
x = tf.keras.layers.Conv2D(64, 3, activation='relu', padding='same')(x)
x = tf.keras.layers.Conv2D(64, 3, activation='relu', padding='same')(x)

# Add a flatten layer to convert the 2D convolutional feature maps into a 1D feature vector
x = tf.keras.layers.Flatten()(x)

# Add a dense layer to map the feature vector to the latent space
z_mean = tf.keras.layers.Dense(200)(x)
z_log_var = tf.keras.layers.Dense(200)(x)

# Add the latent_sampling layer
z = latent_sampling()(z_mean, z_log_var)

# Create the encoder model
encoder = tf.keras.Model(inputs=enc_input, outputs=z)

# Encode an input tensor
inputs = tf.random.normal((32, 128, 128, 3))
latent = encoder(inputs)



In [4]:
### Create the decoder model, using the functional API and the architecture
### detailed below. Use tf.keras.models.Model to initialise the model.

# Model: "decoder"
# ____________________________________________________________________________________________________
#  Layer (type)                   Output Shape           Activation  kernel_size  padding  Input
# ====================================================================================================
#  dec_input (InputLayer)         [(None, 200)]          None
#  dec_dense (Dense)              (None, 4096)           ReLU        None         None     dec_input
#  dec_reshape (Reshape)          (None, 8, 8, 64)       None        None         None     dec_dense
#  dec_conv_1 (Conv2DTranspose)   (None, 8, 8, 64)       ReLU        (3,3)        'same'   dec_reshape
#  dec_conv_2 (Conv2DTranspose)   (None, 16, 16, 64)     ReLU        (3,3)        'same'   dec_conv_1
#  dec_conv_3 (Conv2DTranspose)   (None, 32, 32, 64)     ReLU        (3,3)        'same'   dec_conv_2    
#  dec_conv_4 (Conv2DTranspose)   (None, 64, 64, 32)     ReLU        (3,3)        'same'   dec_conv_3    
#  dec_output (Conv2DTranspose)   (None, 128, 128, 3)    ReLU        (3,3)        'same'   dec_conv_4

from tensorflow.keras.layers import Input, Conv2D, Flatten, Dense, Conv2DTranspose, Reshape

#dec_input = Input(shape=(200,))
#dec_dense = Dense(units=4096, activation='relu')(dec_input)
#dec_reshape = Reshape(target_shape=(8, 8, 64))(dec_dense)
#dec_conv_1 = Conv2DTranspose(filters=64, kernel_size=(3, 3), padding='same', activation='relu')(dec_reshape)
#dec_conv_2 = Conv2DTranspose(filters=64, kernel_size=(3, 3), padding='same', activation='relu')(dec_conv_1)
#dec_conv_3 = Conv2DTranspose(filters=64, kernel_size=(3, 3), padding='same', activation='relu')(dec_conv_2)
#dec_conv_4 = Conv2DTranspose(filters=32, kernel_size=(3, 3), padding='same', activation='relu')(dec_conv_3)
#dec_output = Conv2DTranspose(filters=3, kernel_size=(3, 3), padding='same', activation='relu')(dec_conv_4)
#decoder_model = Model(inputs=dec_input, outputs=dec_output)

# Define the input layer for the decoder
dec_input = tf.keras.layers.Input(shape=(200,))

# Add a sequence of dense and reshape layers to the decoder
x = tf.keras.layers.Dense(4096, activation='relu')(dec_input)
x = tf.keras.layers.Reshape((8, 8, 64))(x)

# Add a sequence of convolutional layers to the decoder
x = tf.keras.layers.Conv2DTranspose(64, 3, activation='relu', padding='same')(x)
x = tf.keras.layers.Conv2DTranspose(64, 3, activation='relu', padding='same')(x)
x = tf.keras.layers.Conv2DTranspose(64, 3, activation='relu', padding='same')(x)
x = tf.keras.layers.Conv2DTranspose(32, 3, activation='relu', padding='same')(x)

# Add the output layer for the decoder
dec_output = tf.keras.layers.Conv2DTranspose(3, 3, activation='relu', padding='same')(x)

# Create the decoder model
decoder = tf.keras.Model(inputs=dec_input, outputs=dec_output)



In [1]:
### Create the VAE model, again using tf.keras.models.Model, with the function
### API to combine the feed the outputs of the encoder into the inputs of the
### decoder.

#Import the necessary layers from TensorFlow:
from tensorflow.keras.layers import Input, Conv2D, Flatten, Dense, Conv2DTranspose, Reshape

#Define the input layer for the encoder model by using the Input function and specify the shape of the input tensor:
enc_input = Input(shape=(128, 128, 3))

# Modify the number of filters in the Conv2D layers in the encoder so that the final feature map has a shape of [batch_size, 2, 2, 1600]
x = Conv2D(32, 3, activation='relu', padding='same')(enc_input)
x = Conv2D(64, 3, activation='relu', padding='same')(x)
x = Conv2D(64, 3, activation='relu', padding='same')(x)
x = Conv2D(64, 3, activation='relu', padding='same')(x)
x = Conv2D(1600, 3, activation='relu', padding='same')(x)

# Flatten the final feature map to convert it into a 1D feature vector
x = Flatten()(x)

# Map the feature vector to the latent space
z_mean = Dense(50)(x)
z_log_var = Dense(50)(x)

# Sample from the latent space
z = latent_sampling()(z_mean, z_log_var)

# Initialize the decoder model
dec_input = Input(shape=(50,))
dec_dense = Dense(units=1600, activation='relu')(dec_input)

# Reshape the decoder output to the desired shape [batch_size, 2, 2, 1600]
dec_reshape = x = Reshape(target_shape=(2, 2, 1600))(x)

# Add the transpose convolutional layers to the decoder model
x = Conv2DTranspose(filters=64, kernel_size=(3, 3), padding='same', activation='relu')(x)
x = Conv2DTranspose(filters=64, kernel_size=(3, 3), padding='same', activation='relu')(x)
x = Conv2DTranspose(filters=64, kernel_size=(3, 3), padding='same', activation='relu')(x)
x = Conv2DTranspose(filters=64, kernel_size=(3, 3), padding='same', activation='relu')(x)
x = Conv2DTranspose(filters=32, kernel_size=(3, 3), padding='same', activation='relu')(x)

# Add the output layer to the decoder model
outputs = Conv2DTranspose(filters=3, kernel_size=(3, 3), padding='same', activation='relu')(x)

# Initialize the encoder and decoder models
encoder = Model(inputs=enc_input, outputs=z)
decoder = Model(inputs=dec_input, outputs=outputs)

# Use the encoder model to map the input tensor to the latent space
latent = encoder(inputs)

# Use the decoder model to map the latent tensor back to the original space
outputs = decoder(latent)

# Initialize the VAE model
vae = Model(inputs=inputs, outputs=outputs)






ResourceExhaustedError: ignored

# Task 3 - Train the model

In [ ]:
# Provided here are the loss functions for the VAE model.

def recon_loss(y_true, y_pred):
    recon = tf.reduce_sum(tf.square(y_true-y_pred), axis=(1,2,3))
    return tf.reduce_mean(recon)

In [ ]:
# This code is provided to load a subsample of the celeb_a dataset, and process
# the images into the correct format for the model.


def img_process(features):
    """
    A preprocessing fuction for the test and validation datasets. This function
    accepts the oxford_iiit_pet dataset, extracts the images and species label,
    and resizes and rescales the images.
    """
    image = tf.image.resize(features['image'], (128,128))
    image = tf.cast(image, 'float32')/255.
    return image, image


train_ds, test_ds = tfds.load('celeb_a', split=['train[:10%]', 'test[:10%]'], download=True, shuffle_files=True)
train_ds = train_ds.map(img_process).cache().batch(64)
test_ds = test_ds.map(img_process).cache().batch(64)


In [ ]:
### Compile the VAE model, choosing an appropriate optimizer and learning rate,
### the total_loss function as the model loss, and any appropriate metrics.

#optimizer: This argument specifies the optimization algorithm to use for training the model. You can choose an appropriate optimizer such as Adam, SGD, or RMSprop, depending on your specific model and task. For example, you can use the Adam optimizer as follows:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

#loss: This argument specifies the loss function to use for training the model. In the case of a VAE model, you can use the total loss function, which is the sum of the reconstruction loss and the KL divergence loss. For example:
loss = total_loss

#metrics: This argument specifies the metrics to use for evaluating the model. You can choose any appropriate metrics such as accuracy, precision, or AUC, depending on your specific model and task. For example:
metrics = ['accuracy']

#With these arguments, you can compile the VAE model as follows
vae.compile(optimizer=optimizer, loss=loss, metrics=metrics)

#This will compile the VAE model, using the specified optimizer, loss function, and metrics. You can then use the compiled model to fit the training data, evaluate the model on the test data, and make predictions on new data.

In [ ]:
### Train the model using the train dataset for an appropriate number of epochs.
### Store the losses and metrics in the history dictionary.

#You can store the losses and metrics in the history dictionary by setting the history argument of the fit method to a dictionary object
history = {'loss': [], 'accuracy': []}

#With these arguments, you can train the VAE model as follows:
history = model.fit(x=train_ds, y=train_ds, batch_size=64, epochs=10, 
validation_data=(test_ds, test_ds), callbacks=None, history=history)



#This will train the VAE model on the training dataset for 10 epochs, using a batch size of 64 and storing the losses and metrics in the history dictionary. You can then access the history of the training process by using the history attribute of the Model object. For example, you can access the loss values for each epoch as follows:
loss_values = history.history['loss']

#You can also access the metric values for each epoch as follows:
accuracy_values = history.history['accuracy']

#This will allow you to plot the training and validation losses and metrics over time, and monitor the progress of the model during training


# Task 4 - Analyse the Model

In [ ]:
### Plot the losses and metrics. Comment on the figures in your report, with
### regard to how the training has proceeded.

#To plot the losses and metrics of the VAE model, you can use the matplotlib library in Python. Here is an example of how you can do this:
import matplotlib.pyplot as plt

# Plot the training and validation losses
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Training Loss', 'Validation Loss'], loc='upper right')
plt.show()

# Plot the training and validation metrics
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Training Accuracy', 'Validation Accuracy'], loc='lower right')
plt.show()

#


In [ ]:
### Using the test dataset, create a plot that shows the reconstruction quality
### of the training model. Comment on the results in your report.

# Generate reconstructions of the test images
reconstructions = vae.predict(test_ds)

# Select a random subset of the test images and reconstructions
num_samples = 10
indices = np.random.randint(0, len(test_ds), num_samples)
test_images = test_ds[indices]
recon_images = reconstructions[indices]

# Plot the test images and reconstructions
plt.figure(figsize=(10, 10))
for i in range(num_samples):
    plt.subplot(num_samples, 2, 2*i+1)
    plt.imshow(test_images[i])
    plt.axis('off')
    plt.title('Original')
    plt.subplot(num_samples, 2, 2*i+2)
    plt.imshow(recon_images[i])
    plt.axis('


In [ ]:
### Demonstrate the generative properties of the VAE by drawing randomly sampled
### latent vectors from a unit Guassian and passing them to the train decoder.
### Plot the results and comment on them in your report.

# Draw a random latent vector from a unit Gaussian
latent_vector = np.random.normal(size=(1, 200))

# Generate a new image from the latent vector
generated_image = decoder.predict(latent_vector)

# Plot the generated image
plt.imshow(generated_image[0])
plt.axis('off')
plt.title('Generated Image')
plt.show()

#You can repeat this process multiple times to generate a set of new images, and plot them together to visualize the generative capabilities of the VAE model.
# Draw a set of latent vectors from a unit Gaussian
latent_vectors = np.random.normal(size=(10, 200))

# Generate a set of new images from the latent vectors
generated_images = decoder.predict(latent_vectors)

# Plot the generated images
plt.figure(figsize=(10, 10))
for i in range(10):
    plt.subplot(5, 5, i+1)
    plt.imshow(generated_images[i])
    plt.axis('off')
plt.show()
